# HiFive Unmatched Power Monitor Data Viewer

In [ ]:
import datetime
import nptdms
import numpy
import pandas
import pathlib

from bokeh.models import Legend
from bokeh.palettes import Category20
from bokeh.plotting import figure
from bokeh.plotting import output_notebook
from bokeh.plotting import show

## Functions

In [ ]:
def load_measurements(
        tdms_path,
        tdms_group_name,
        power_pins):
    """Load current measurements from a input TDMS file.
    
    Load current measurements and channel metadata from the input TDMS
    file, then compute power traces.
    """
    with nptdms.TdmsFile.open(tdms_path) as tdms_file:
        # Check the expected measurement group is present in the
        # input TDMS file.
        found = False
        for group in tdms_file.groups():
            if tdms_group_name == group.name:
                found = True
        if not found:
            raise ValueError(f"TDMS group `{tdms_group_name}` not found")

        # Load current measurements.
        current = tdms_file[tdms_group_name].as_dataframe(
            time_index=True,
            absolute_time=False
        )
            
        # Load channel metadata.
        metadata = {
            pin: tdms_file[tdms_group_name][pin].properties
            for pin in power_pins
        }
        
        # Compute power traces.
        power = pandas.DataFrame(
            data={
                pin: current[pin] * vdd
                for pin, vdd in power_pins.items()
            },
            index=current.index
        )

    return current, metadata, power


def average_power_consumption(
        power,
        time_interval):
    """Average power consumption over time windows.
    """
    # Compute number of time windows of the specified length in
    # the power trace.
    n_intervals = numpy.uint(power.index.max() / time_interval)
    
    # Compute average power consumption for each time interval.
    average = pandas.DataFrame(
        data=[
            power[i*time_interval:(i+1)*time_interval].mean()
            for i in range(n_intervals)
        ],
        index=[
            i * time_interval
            for i in range(n_intervals)
        ]
    )
    
    return average


def normalize_power_consumption(power):
    """Normalize power consumption components.
    """
    overall = power.sum(axis=1)
    normalized = pandas.DataFrame(
        data={
            col: power[col] / overall
            for col in power
        },
        index=power.index
    )
    
    return normalized


def make_plot_figure(
        plot_width,
        plot_height,
        plot_xlabel,
        plot_ylabel):
    """Create a figure to plot data into.
    """
    # Create figure.
    fig = figure(
        plot_width=plot_width,
        plot_height=plot_height,
        tools="box_zoom,save,reset",
        toolbar_location="below"
    )
    
    # Configure axis labels.
    fig.xaxis.axis_label = plot_xlabel
    fig.xaxis.axis_label_text_font = "Arial"
    fig.xaxis.axis_label_text_color = "black"
    fig.xaxis.axis_label_text_font_size = "11pt"
    fig.xaxis.axis_label_text_font_style = "bold"
    fig.yaxis.axis_label = plot_ylabel
    fig.yaxis.axis_label_text_font = "Arial"
    fig.yaxis.axis_label_text_color = "black"
    fig.yaxis.axis_label_text_font_size = "11pt"
    fig.yaxis.axis_label_text_font_style = "bold"
    
    # Configure major labels.
    fig.xaxis.major_label_text_color = "black"
    fig.xaxis.major_label_text_font = "Arial"
    fig.xaxis.major_label_text_font_size = "11pt"
    fig.yaxis.major_label_text_color = "black"
    fig.yaxis.major_label_text_font = "Arial"
    fig.yaxis.major_label_text_font_size = "11pt"

    # Create legend.
    legend = Legend(
        label_text_font="Arial",
        label_text_font_size="9pt",
        label_text_color="black",
        border_line_color="black",
        border_line_alpha=1,
        border_line_width=1,
        click_policy="hide"
    )
    fig.add_layout(legend, "right")
    
    return fig


def plot_power_signals(
        traces,
        plot_width=950,
        plot_height=350):
    """Plot all power traces in the same figure.
    """
    fig =  make_plot_figure(
        plot_width,
        plot_height,
        "Time [s]",
        "Power [W]"
    )
    palette = Category20[traces.columns.size]
    for i, column in enumerate(traces.columns):
        fig.line(
            "index",
            column,
            source=traces,
            color=palette[i],
            legend_label=column,
            line_width=1.5
        )

    return fig


def plot_power_contributions_vbar(
        power,
        plot_width=950,
        plot_height=350):
    """
    """
    fig =  make_plot_figure(
        plot_width,
        plot_height,
        "Time [s]",
        "Power [W]"
    )
    palette = Category20[power.columns.size]
    fig.vbar_stack(
        power.columns.to_list(),
        x="index",
        source=power,
        color=palette,
        width=(power.index[1] - power.index[0]) * 0.8,
        legend_label=power.columns.to_list(),
        line_color="black",
        fill_alpha=0.75
    )
    
    return fig


def plot_power_contributions_varea(
        power,
        plot_width=950,
        plot_height=350):
    """
    """
    fig =  make_plot_figure(
        plot_width,
        plot_height,
        "Time [s]",
        "Power [W]"
    )
    palette = Category20[power.columns.size]
    fig.varea_stack(
        power.columns.to_list(),
        x='index',
        color=palette,
        legend_label=power.columns.to_list(),
        source=power,
        fill_alpha=0.75
    )

    return fig

## Experiment Viewer

In [ ]:
# Power measurements.
POWER_MEASUREMENT_PINS = {
    "i_core":    0.918956,
    "i_ddr_soc": 1.197213,
    "i_io":      1.793037,
    #"i_pll":     1.794147,
    "i_pcievp":  0.923521,
    "i_pcievph": 1.792846,
    "i_ddr_mem": 1.197322,
    "i_ddr_pll": 0.919675,
    "i_ddr_vpp": 2.442895
}
POWER_AVG_TIME_COARSE = 1000e-3
POWER_AVG_TIME_FINE = 1000e-6

# Experiment information.
EXPERIMENT_TDMS_PATHS = {
    "HiFiveUnmatched_boot": (
        pathlib.Path("experiments\\HiFiveUnmatched_boot.tdms").resolve(),
        "PXIe-4309"
    ),
    "HiFiveUnmatched_idle": (
        pathlib.Path("experiments\\HiFiveUnmatched_idle.tdms").resolve(),
        "PXIe-4309"
    ),
    "HiFiveUnmatched_aes128cbc": (
        pathlib.Path("experiments\\HiFiveUnmatched_aes128cbc.tdms").resolve(),
        "PXIe-4309"
    ),
    "HiFiveUnmatched_aes128ctr": (
        pathlib.Path("experiments\\HiFiveUnmatched_aes128ctr.tdms").resolve(),
        "PXIe-4309"
    ),
    "HiFiveUnmatched_scpaes128ctr100mb": (
        pathlib.Path("experiments\\HiFiveUnmatched_scpaes128ctr100mb.tdms").resolve(),
        "PXIe-4309"
    ),
    "HiFiveUnmatched_hpl": (
        pathlib.Path("experiments\\HiFiveUnmatched_hpl.tdms").resolve(),
        "PXIe-4309"
    ),
    "HiFiveUnmatched_stream": (
        pathlib.Path("experiments\\HiFiveUnmatched_stream.tdms").resolve(),
        "PXIe-4309"
    )
}
for name, (path, _) in EXPERIMENT_TDMS_PATHS.items():
    if not path.is_file():
        raise ValueError(f"Experiment {name} ({path}) NOT found")
        
# Load experiment data.
EXPERIMENT_DATA = {}
for name, (path, group_name) in EXPERIMENT_TDMS_PATHS.items():
    print(f"[{datetime.datetime.now()}] Process experiment `{name}`")
    print(f"[{datetime.datetime.now()}] \tload data")
    current, meta, power = load_measurements(
        path,
        group_name,
        POWER_MEASUREMENT_PINS
    )
    print(f"[{datetime.datetime.now()}] \taverage power consumption (coarse-grain)")
    power_avg_coarse = average_power_consumption(
        power,
        POWER_AVG_TIME_COARSE
    )
    print(f"[{datetime.datetime.now()}] \taverage power consumption (fine-grain)")
    power_avg_fine = average_power_consumption(
        power,
        POWER_AVG_TIME_FINE
    )
    EXPERIMENT_DATA[name] = {
        "meta": meta,
        "current": current,
        "power": power,
        "power_avg_coarse": power_avg_coarse,
        "power_avg_fine": power_avg_fine
    }

In [ ]:
output_notebook()

show(plot_power_signals(EXPERIMENT_DATA["HiFiveUnmatched_stream"]["power_avg_fine"]))
show(plot_power_contributions_vbar(EXPERIMENT_DATA["HiFiveUnmatched_stream"]["power_avg_coarse"]))
show(plot_power_contributions_varea(EXPERIMENT_DATA["HiFiveUnmatched_stream"]["power_avg_fine"]))